In [21]:
from telethon.sync import TelegramClient
from client import  parse_channel_messages
from config import API_ID, API_HASH, TOKEN

import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

/home/daniil/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = TelegramClient('session_name', API_ID, API_HASH)

In [18]:
ans = await parse_channel_messages(client, 'cbrstocks', limit=5)

In [19]:
ans = ans[2:-1]

In [20]:
ans

['#MGNT #Идея\n**🧠**** **[Шпион РЦБ](https://t.me/stockspy_bot)** указывает на возросший интерес крупных покупателей в акциях Магнита**\n\nВ последние дни видим аномальные объемы на покупку от крупных игроков, включая авторов закрытых каналов. \n\n**Мозговой центр Сигналов подтверждает актуальность **[**идеи по Магниту**](https://t.me/cbrstocks/58564)\nТаргет: 10 000 руб/акцию\nТекущая доходность +6%\n\n**Ближайшим триггером к росту акций Магнита может стать объявление дивиденда.**\n\n[👉 Подключиться к Шпиону](https://t.me/stockspy_bot)',
 '🔝 **Сигналы РЦБ снова в числе лучших**\n\nСервис Invest-idei.ru объявил победителей [конкурса](https://invest-idei.ru/contests-for-analysts/options-2024) аналитики "**Опционы на миллионы"**, который проходил при поддержке ПАО Московская Биржа. \n\n__Для победы в конкурсе с декабря 2023 по март 2024 аналитикам было необходимо опубликовать не менее 5 идей на опционы, либо на активы, которые являются базовым активом опционов на Московской Бирже. Всего 

In [22]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment', return_dict=True)


tokenizer_config.json: 100%|████████████████████| 495/495 [00:00<00:00, 555kB/s]
vocab.txt: 100%|███████████████████████████| 1.40M/1.40M [00:01<00:00, 1.30MB/s]
config.json: 100%|██████████████████████████████| 952/952 [00:00<00:00, 771kB/s]
model.safetensors: 100%|█████████████████████| 711M/711M [06:34<00:00, 1.80MB/s]


In [25]:
for text in ans:
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    print(text)
    print(predicted)

#MGNT #Идея
**🧠**** **[Шпион РЦБ](https://t.me/stockspy_bot)** указывает на возросший интерес крупных покупателей в акциях Магнита**

В последние дни видим аномальные объемы на покупку от крупных игроков, включая авторов закрытых каналов. 

**Мозговой центр Сигналов подтверждает актуальность **[**идеи по Магниту**](https://t.me/cbrstocks/58564)
Таргет: 10 000 руб/акцию
Текущая доходность +6%

**Ближайшим триггером к росту акций Магнита может стать объявление дивиденда.**

[👉 Подключиться к Шпиону](https://t.me/stockspy_bot)
[0]
🔝 **Сигналы РЦБ снова в числе лучших**

Сервис Invest-idei.ru объявил победителей [конкурса](https://invest-idei.ru/contests-for-analysts/options-2024) аналитики "**Опционы на миллионы"**, который проходил при поддержке ПАО Московская Биржа. 

__Для победы в конкурсе с декабря 2023 по март 2024 аналитикам было необходимо опубликовать не менее 5 идей на опционы, либо на активы, которые являются базовым активом опционов на Московской Бирже. Всего в турнире приняли